# 🤖 Colab Webhook Server for Automated Transcription

This notebook runs a webhook server that GitHub Actions can trigger automatically.

## ⚠️ Important:
- Keep this notebook running in your browser
- Colab free tier sessions last ~12 hours
- You'll need to restart after session expires

## 🔧 Setup Steps:
1. Run all cells in this notebook
2. Copy the webhook URL that appears
3. Add it to GitHub Secrets as `COLAB_WEBHOOK_URL`
4. Push audio files - transcription happens automatically!

---

## Step 1: Install Dependencies

In [ ]:
!pip install -q flask pyngrok openai-whisper gitpython
!apt-get -qq install -y ffmpeg
print("✓ Dependencies installed!")

## Step 2: Configuration

In [ ]:
# GitHub Configuration
GITHUB_REPO = "kevinzjpeng/voice-record"  # Your repository
GITHUB_TOKEN = ""  # Optional: Add token for private repos or auto-commit

# Webhook Security (recommended)
WEBHOOK_SECRET = "your-secret-key-here"  # Change this!

print("✓ Configuration set")
print(f"  Repository: {GITHUB_REPO}")
print(f"  Security: {'Enabled' if WEBHOOK_SECRET else 'Disabled (not recommended)'}")

## Step 3: Load Whisper Model

In [ ]:
import whisper
import torch

print("Loading Whisper Large model...")
print(f"GPU available: {torch.cuda.is_available()}")

model = whisper.load_model("large")
print("✓ Model loaded successfully!")

## Step 4: Define Transcription Function

In [ ]:
import os
import git
from pathlib import Path

def format_timestamp(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{secs:02d}"

def transcribe_repo_files(repo_url, audio_files):
    """
    Clone repo, transcribe audio files, and commit results.
    """
    try:
        # Clone repository
        print(f"Cloning {repo_url}...")
        if os.path.exists('repo'):
            import shutil
            shutil.rmtree('repo')
        
        repo = git.Repo.clone_from(repo_url, 'repo')
        os.chdir('repo')
        
        # Transcribe each audio file
        transcripts = []
        for audio_file in audio_files:
            if not os.path.exists(audio_file):
                print(f"⚠️  File not found: {audio_file}")
                continue
            
            print(f"\n{'='*60}")
            print(f"Transcribing: {audio_file}")
            print(f"{'='*60}")
            
            result = model.transcribe(
                audio_file,
                language='zh',
                task='transcribe',
                verbose=True
            )
            
            # Save transcript
            transcript_path = Path(audio_file).with_suffix('.txt')
            with open(transcript_path, 'w', encoding='utf-8') as f:
                f.write(f"Transcript of: {Path(audio_file).name}\n")
                f.write(f"Language: Cantonese/Chinese\n")
                f.write(f"{'='*60}\n\n")
                f.write(result['text'].strip())
                f.write("\n\n")
                f.write(f"{'='*60}\n")
                f.write("Detailed segments:\n\n")
                
                for segment in result['segments']:
                    start = format_timestamp(segment['start'])
                    end = format_timestamp(segment['end'])
                    text = segment['text'].strip()
                    f.write(f"[{start} -> {end}] {text}\n")
            
            transcripts.append(str(transcript_path))
            print(f"✓ Saved: {transcript_path}")
        
        # Commit and push results
        if transcripts and GITHUB_TOKEN:
            repo.git.add('voice-record/**/*.txt')
            repo.index.commit('Add Cantonese transcripts [automated]')
            repo.remote('origin').push()
            print("\n✓ Transcripts committed and pushed!")
        
        os.chdir('..')
        return {"status": "success", "transcripts": transcripts}
        
    except Exception as e:
        print(f"✗ Error: {str(e)}")
        return {"status": "error", "message": str(e)}

print("✓ Transcription function ready")

## Step 5: Start Webhook Server

🚨 **This cell will run continuously. Don't stop it!**

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading
import hashlib
import hmac

app = Flask(__name__)

def verify_signature(payload, signature):
    """Verify webhook signature for security."""
    if not WEBHOOK_SECRET:
        return True  # Skip verification if no secret set
    
    expected = hmac.new(
        WEBHOOK_SECRET.encode(),
        payload,
        hashlib.sha256
    ).hexdigest()
    
    return hmac.compare_digest(f"sha256={expected}", signature)

@app.route('/health', methods=['GET'])
def health():
    """Health check endpoint."""
    return jsonify({
        "status": "healthy",
        "model": "whisper-large",
        "gpu": torch.cuda.is_available()
    })

@app.route('/transcribe', methods=['POST'])
def transcribe():
    """Webhook endpoint for transcription."""
    
    # Verify signature
    signature = request.headers.get('X-Hub-Signature-256', '')
    if not verify_signature(request.data, signature):
        return jsonify({"error": "Invalid signature"}), 403
    
    data = request.json
    repo_url = data.get('repository_url', f"https://github.com/{GITHUB_REPO}.git")
    audio_files = data.get('audio_files', [])
    
    print(f"\n{'='*60}")
    print(f"Webhook triggered!")
    print(f"Repository: {repo_url}")
    print(f"Audio files: {audio_files}")
    print(f"{'='*60}\n")
    
    # Start transcription in background
    def background_transcribe():
        result = transcribe_repo_files(repo_url, audio_files)
        print(f"\n✓ Transcription complete: {result}")
    
    thread = threading.Thread(target=background_transcribe)
    thread.start()
    
    return jsonify({
        "status": "processing",
        "message": "Transcription started"
    })

# Start ngrok tunnel
port = 5000
public_url = ngrok.connect(port)

print("\n" + "="*70)
print("🎉 WEBHOOK SERVER RUNNING!")
print("="*70)
print(f"\n📍 Public URL: {public_url}")
print(f"\n🔗 Webhook endpoint: {public_url}/transcribe")
print(f"\n💡 Add this to GitHub Secrets as COLAB_WEBHOOK_URL:")
print(f"   {public_url}/transcribe")
print("\n" + "="*70)
print("\n⚠️  Keep this cell running! Don't stop it.")
print("\n✅ Ready to receive transcription requests!\n")

# Run Flask app
app.run(port=port)

---

## 🎉 Server Running!

### What's happening?
- Webhook server is running on Colab
- ngrok provides a public URL
- GitHub Actions can now trigger this server

### To complete setup:
1. Copy the webhook URL from above
2. Go to your GitHub repository
3. Settings → Secrets and variables → Actions
4. Add new secret: `COLAB_WEBHOOK_URL` = your webhook URL
5. (Optional) Add secret: `WEBHOOK_SECRET` = your security key

### Now test it:
```bash
git add voice-record/test.mp3
git commit -m "Test automated transcription"
git push
```

Watch the output below - you'll see the transcription happen automatically!

### ⚠️ Important:
- Keep this notebook tab open
- Don't stop the cell above
- Session expires after ~12 hours (restart if needed)